In [23]:
import transformers
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

In [8]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
le = LabelEncoder()

# Sub-task A

In [4]:
train_dataset = pd.read_csv('datasets/train_level_a.csv')
train_dataset

,id,tweet,subtask_a
0,86426,@USER She should ask a few native Americans wh...,OFF
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF
2,16820,Amazon is investigating Chinese employees who ...,NOT
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT
...,...,...,...
13235,95338,@USER Sometimes I get strong vibes from people...,OFF
13236,67210,Benidorm ✅ Creamfields ✅ Maga ✅ Not too sh...,NOT
13237,82921,@USER And why report this garbage. We don't g...,OFF
13238,27429,@USER Pussy,OFF


In [9]:
tokenized_data = tokenizer(train_dataset["tweet"].tolist(), return_tensors="np", padding=True)
tokenized_data = dict(tokenized_data)
labels = np.array(le.fit_transform(train_dataset["subtask_a"]))

In [17]:
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'NOT': 0, 'OFF': 1}


In [10]:
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model.compile(optimizer=Adam(3e-5))
model.fit(tokenized_data, labels)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


414/414 [==============================] - 15853s 38s/step - loss: 0.4730


In [11]:
model.save_pretrained("models/model_subtask_a")

In [12]:
test_dataset = pd.read_csv('datasets/test_level_a.csv')
test_dataset

,id,tweet,label
0,15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
1,27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
2,30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
3,13876,#Watching #Boomer getting the news that she is...,NOT
4,60133,#NoPasaran: Unity demo to oppose the far-right...,OFF
...,...,...,...
855,73439,#DespicableDems lie again about rifles. Dem Di...,OFF
856,25657,#MeetTheSpeakers 🙌 @USER will present in our e...,NOT
857,67018,3 people just unfollowed me for talking about ...,OFF
858,50665,#WednesdayWisdom Antifa calls the right fascis...,NOT


In [26]:
tweets = test_dataset['tweet']
y_true = le.fit_transform(test_dataset['label'])

In [20]:
tokenized = tokenizer(tweets.tolist(), return_tensors="np", padding="longest")
outputs = model(tokenized).logits

y_pred = np.argmax(outputs, axis=1)

In [28]:
print(accuracy_score(y_true, y_pred))
print()
print(classification_report(y_true, y_pred))

0.8348837209302326

              precision    recall  f1-score   support

           0       0.87      0.90      0.89       620
           1       0.73      0.65      0.69       240

    accuracy                           0.83       860
   macro avg       0.80      0.78      0.79       860
weighted avg       0.83      0.83      0.83       860



# Sub-task B

# Sub-task C